In [2]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sentence_transformers import SentenceTransformer, util
import numpy as np

# --- Load intents.json ---
with open("intents.json", "r", encoding="utf-8") as f:
    intents_data = json.load(f)

patterns = []
labels = []

for intent in intents_data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        labels.append(intent['intent'])

# Split train/test (optional)
X_train, X_test, y_train, y_test = train_test_split(patterns, labels, test_size=0.2, random_state=42)

# --- Train Intent Classifier ---
intent_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(max_iter=500))
])
intent_clf.fit(X_train, y_train)

print(f"Intent Classifier Accuracy: {intent_clf.score(X_test, y_test)*100:.2f}%")



Intent Classifier Accuracy: 64.29%


In [3]:
# Packages
packages = pd.read_csv("packages.csv")
packages['type'] = packages['type'].str.lower()
packages['destination'] = packages['destination'].str.lower()

# FAQs
faqs = pd.read_csv("faq.csv")
faqs['question'] = faqs['question'].str.lower()
faqs['answer'] = faqs['answer']


In [4]:
# Load SentenceTransformer model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Package embeddings (combine type + destination + description)
package_texts = (packages['type'] + " " + packages['destination'] + " " + packages['description']).tolist()
package_embeddings = embed_model.encode(package_texts, convert_to_tensor=True)

# FAQ embeddings
faq_texts = faqs['question'].tolist()
faq_embeddings = embed_model.encode(faq_texts, convert_to_tensor=True)


In [10]:
def get_response(user_input):
    text = user_input.lower().strip()
    
    # Predict intent first
    intent = intent_clf.predict([text])[0]

    # Handle intents explicitly
    if intent == 'greet':
        return random.choice(["Hi! How can I help you plan your trip?", "Hello! Ready to explore destinations?"])
    if intent == 'bye':
        return random.choice(["Goodbye! Have a great trip!", "See you soon!"])
    
    # FAQ retrieval
    if intent == 'ask_faq':
        user_emb = embed_model.encode(text, convert_to_tensor=True)
        cos_scores = util.cos_sim(user_emb, faq_embeddings)[0]
        top_idx = torch.argmax(cos_scores).item()  # Convert tensor to int
        return f"Here's what I found:\n- {faqs.iloc[top_idx]['answer']}"

    # Package recommendation
    if intent == 'recommend':
        user_emb = embed_model.encode(text, convert_to_tensor=True)
        cos_scores = util.cos_sim(user_emb, package_embeddings)[0]
        top_results = torch.topk(cos_scores, k=min(3, len(packages)))
        top_indices = top_results.indices.tolist()  # Convert to list
        response = "You might love these destinations:"
        for idx in top_indices:
            pkg = packages.iloc[idx]
            response += f"\n- **{pkg['destination']}**: A {pkg['type']} trip. {pkg['description']} (Approx. ₹{pkg['price']})"
        return response

    # Booking intent
    if intent == 'book':
        return "Sure! Which destination would you like to book, and for how many travellers?"

    # Fallback
    return "I'm sorry, I don't understand. You can ask me to recommend trips, answer FAQs, or help you book a package."


In [11]:
def start_chat():
    print("Travelounce AI Assistant (Model-based)")
    print("Type 'bye' to exit.\n")
    print("Bot: Hi! Ready to plan your next trip?\n")

    while True:
        user_input = input("You: ").strip()
        if not user_input:
            continue
        response = get_response(user_input)
        print(f"\nBot: {response}\n")
        if intent_clf.predict([user_input])[0] == 'bye':
            break

if __name__ == "__main__":
    start_chat()


Travelounce AI Assistant (Model-based)
Type 'bye' to exit.

Bot: Hi! Ready to plan your next trip?


Bot: Hi! How can I help you plan your trip?


Bot: You might love these destinations:
- **goa**: A beach trip. 3 Nights stay with water sports and breakfast (Approx. ₹12000)
- **andaman**: A beach trip. 4 Nights Havelock + Neil Island trip (Approx. ₹30000)
- **maldives**: A honeymoon trip. Water villa stay with private beach (Approx. ₹65000)


Bot: Here's what I found:
- Refunds are processed within 5–7 business days depending on your payment method.


Bot: You might love these destinations:
- **rishikesh**: A adventure trip. Camping + rafting package for 2 nights (Approx. ₹8000)
- **ladakh**: A adventure trip. 5 Days bike trip with camping (Approx. ₹35000)
- **manali**: A adventure trip. Paragliding, trekking, rafting included (Approx. ₹18000)


Bot: Sure! Which destination would you like to book, and for how many travellers?


Bot: Here's what I found:
- Most packages include breakfas

KeyboardInterrupt: Interrupted by user